# 3.8 Applications of Locality-Sensitive Hashing

One subject covered is Entity resolution (p. 110)

    1. Entity Resolution refers to matching data records that refer to the same real-world entity, e.g. the
    same person. The principal problem addressed here is that the similarity of records does not match exactly either 
    the similar-sets or similar-vectors models of similarity on which the theory is built. 
    
## 3.8.2 An Entity-Resolution Example

Here I'm going to replicated the entity resolution example given in MMDS. The context is that Company A disagree with Company B over how many customers A had provided to B in the form of leads; each company had about 1 million records. The problem was to match records from the two sets to see if a pair represented the same person. Key facts about the data:

- Each record had fields for the name, address and phone number of the person.
- Names could be mispelled, phone numbers could be different, area codes of phones sometimes change, etc.

The strategy for identifying records involved scoring the differences in three fields: name, address, and phone. To create a <i>score</i> describing the likelihood that two records, one from A and the other from B, described the same person, 100 points where assigned to each of the three fields, so records with exact matches in all three fields got a score of 300. 

However, there were deductions for misses in each of the fields. As a first approximation, edit-distance (Section 3.5.5) was used, but the penalty grew quadratically with the distance. Then, certain publically available tables were used to reduce the penalty in appropriate situations. For example, "Bill" and "William" were treated as if they differed in only one letter, even though their edit distance is 5.

It is not feasible to score all one trillion pairs of records. Thus, a simple LSH was used to focus on the likely candidates. Three "hash functions" were used. The first sent records to the same bucket only if they had identical names; the second did the same but for identical addresses, and the third did the same for phone numbers. <b>In practice, there was no hashing</b>; rather the records were sorted by name, so records with identical names would appear consecutively and get scored for overall similarity of the name, address, and phone. Then the records were sorted by address, and those with the same address were scored. Finally, the records were sorted a third time by phone, and records with identical phones were scored. 

This approach missed a record pair that truly represented the same person but none of the three fields matched exactly. Since the goal was to prove in a court of law that the persons were the same, it is unlikely that such a pair would have been accepted by a judge as sufficiently similar anyway.

## Replication - Section 3.8.2, Entity Resolution Example

- Data with some variation in names, addresses, and phone numbers
- Locality-Sensitive Hashing function
- Edit distance function
- Penalty table for common abbreviations (William and Bill gets edit distance corrected from 5 to 1)

## Use case

- Instead of 1 million names, we would like this to work with around 300 million names.
- Example data must be publically available

## Tools

- Apache Spark
- Python or Scala; not sure yet.


## Replication Data 

We would like data with *names*, *addresses*, and *phone numbers* with about 300 million records that have been entered by hand. You can't always get what you want but if you try sometimes then you just might find that you get what you [need](https://www.youtube.com/watch?v=XG5GOH2CO1k).

Right now it's a toss up between the [Wikipedia Data](https://code.google.com/p/wiki-links/downloads/list) or the [Federal Contracts from the Federal Procurement Data](http://aws.amazon.com/datasets/federal-contracts-from-the-federal-procurement-data-center-usaspending-gov/). Both seem like they would be large enough. I think Wikipedia is like 2.5 times larger at 500 gb. I'm concerned that wikipedia won't have enough name variation but it's worth taking a closer look.